In [13]:
!pip install bert_score
import bert_score

  Using cached bert_score-0.3.2-py3-none-any.whl (52 kB)
  Using cached transformers-2.8.0-py3-none-any.whl (563 kB)
  Using cached boto3-1.12.45-py2.py3-none-any.whl (128 kB)
  Using cached sentencepiece-0.1.85-cp37-cp37m-macosx_10_6_x86_64.whl (1.1 MB)
Processing /Users/kuly/Library/Caches/pip/wheels/81/94/a4/1cd28c369532c89aaa9b39580100990d74553e550b1942a7dc/sacremoses-0.0.41-py3-none-any.whl
  Using cached tokenizers-0.5.2.tar.gz (64 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Processing /Users/kuly/Library/Caches/pip/wheels/52/98/5c/8d5d7a5321d2de463dfc6f7b32aab8b16e564cfce553b28daf/regex-2020.4.4-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached jmespath-0.9.5-py2.py3-none-any.whl (24 kB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached botocore-1.15.45-py2.py3-none-any.whl (6.1 MB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  ERROR: Command errored out wi

ModuleNotFoundError: No module named 'bert_score'

In [12]:
%%capture
%matplotlib inline
from model_utils import predict_for_test_samples
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from bert_score import score

def test(vocab_path, model_path, training_image_ids_path):
    sample_size = 100
    test_pred_captions = predict_for_test_samples(image_folder_path = image_folder_path,\
                         sample_size = sample_size, vocab_path = vocab_path, model_path = model_path,\
                           training_image_ids_path = training_image_ids_path)

    #load obj
    im_gender_summary = load_obj("im_gender_summary")
    captions_dict = load_obj("captions_dict")

    gt = []
    pred_gender = []
    bleus = []
    for image_id, caption in test_pred_captions.items():
        gt.append(im_gender_summary[image_id]['pred_gt'])
        pred_gender.append(caption_to_gender(caption))
        ref_captions = captions_dict[image_id]
        ref_captions = [c.split() for c in ref_captions]
        bleus.append(bleu.sentence_bleu(ref_captions, caption.split()))

    _, _, F1 = score(ref_captions, caption.split(), bert="bert-base-uncased")

    return confusion_matrix(gt, pred_gender), accuracy_score(gt, pred_gender), bleus.mean(), F1.mean()

ModuleNotFoundError: No module named 'bert_score'

In [ ]:
import glob
import re

results = dict()

model_dirs = glob.glob('./testmodels/*/')
for d in model_dirs:
    # get name
    p = re.compile(r'(?<=testmodels/)(.*)(?=/)')
    name = p.findall(d)[0]
    
    # get paths
    vocab_path = d + 'vocab.pkl'
    model_path = d +'best-model.pkl'
    training_image_ids_path = d +'training_image_ids.pkl'
    
    conf_matrix, accuracy, bleu, bert = test(vocab_path, model_path, training_images_ids_path)
    
    results[name] = dict()
    results[name]['confusion_matrix'] = conf_matrix
    results[name]['accuracy'] = accuracy
    results[name]['bleu'] = bleu
    results[name]['bert_f1'] = bert
    print(f"Evaluated {name}: {results[name]}")